In [1]:
# import SparkContext and SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [2]:
spark = SparkSession.builder\
        .master("local[8]")\
        .appName("scaling")\
        .getOrCreate()

In [3]:
# load the data from data\*.csv with the types specified to [list, string, timestamp, string, int, string]

df = spark.read.csv('data\World2022-04-01.csv', header=False, inferSchema=True)
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')


In [8]:
df

DataFrame[viewers: string, streamer: string, date: timestamp, category: string, viewership: int, language: string]

In [6]:
df.first()

Row(viewers="['0000moonlight', '001133557799', '0025', '003ny', '007dontae', '00bimalicious00', '00jackal', '00rbiit', '01001101t', '03m1ckey', '03march', '04four', '08sunny', '09ivory', '0___________o', '0__oslick', '0_blazed_0', '0_mezo', '0apie', '0ats4gave', '0ava', '0blivi0nnnn', '0bluew', '0dane', '0daysleft', '0ddacity', '0dol', '0ekan', '0flask0', '0g_banana', '0hesty', '0hno21', '0hnohedidnt', '0ldseoul', '0lija', '0liv5r', '0megadance', '0megasoul', '0moaffe0', '0monke', '0mrchow', '0nebri', '0nogard0', '0novae', '0numberz', '0o000ooo', '0o_star_dust_o0', '0ogaboooga', '0ptimal_oct0pus', '0rangehokage', '0rt0ian', '0shin0_', '0siristv', '0slushiestash0', '0smar', '0uthere', '0wolvorine0', '0x78da4f00', '0xexcaliburp', '0xwong', '0xxwhyxx0', '0yassine', '100ahmed', '100kr', '105rookie', '10ajt', '1111infinity1111', '111antoni111', '111unnamed111', '11martin112', '11sidekick11', '123hehexd123xd', '12at12', '12knism', '12nine07', '12sleep3', '1301dominik', '1301pascal', '1336plu